# Practical: Risk Prediction
## Advanced Statistics for Records Research

## Research question

In this session, we will explore the dataset of 2000 participants we met in the lecture, and fit
a risk prediction model for death within 5 years, based on some simple patient
characteristics.

## Objectives
By the end of this practical, you should be able to:

1. Fit a logistic model to create risk predictions.

2. Assess model discrimination by calculating the Area Under the Curve.

3. Assess model calibration by graphing observed and predicted risks.

## Dataset and analysis
For this practical we will use a (simulated) dataset called `data_predict`. This contains
data for 2,000 patients, with information on six variables.

| Variable | Description             |
|----------|-------------------------|
| id       | Unique patient ID       |
| age      | Age (years)             |
| sbp      | Systolic Blood Pressure |
| bmi      | Body Max Index $kg/m^2$ |
| sex      | Female / Male           |
| dead     | Alive / Dead            |


In [34]:
# Read data
import pandas as pd
import numpy as np
data_predict = pd.read_stata("../data/Prediction-data.dta")

In [3]:
# Inspect data
data_predict

,id,age,sbp,bmi,sex,dead
0,1,47.0,116.8,25.6,Female,Alive
1,2,71.0,113.8,21.1,Male,Alive
2,3,41.0,130.7,25.6,Male,Alive
3,4,71.0,118.2,25.9,Male,Alive
4,5,54.0,120.3,20.7,Male,Alive
...,...,...,...,...,...,...
1995,1996,47.0,126.3,29.1,Male,Alive
1996,1997,79.0,131.2,33.0,Male,Alive
1997,1998,46.0,135.0,24.5,Male,Alive
1998,1999,42.0,131.1,22.3,Male,Alive


## Data exploration

Have a look at the data, then try answering the following questions:
* How many participants died at the end of the follow-up?
* What is the proportion of female participants?
* What is the mean, standard deviation, minimum, and maximum age of these participants?

In [4]:
data_predict \
    .groupby('dead') \
    .size().to_frame('n') \
    .assign(percent = lambda x: x.n/sum(x.n)*100)

,n,percent
dead,,
Alive,1491,74.55
Dead,509,25.45


In [5]:
data_predict \
    .groupby('sex') \
    .size().to_frame('n') \
    .assign(percent = lambda x: x.n/sum(x.n)*100)

,n,percent
sex,,
Female,978,48.9
Male,1022,51.1


In [6]:
data_predict[['age', 'sbp', 'bmi']].describe()

,age,sbp,bmi
count,2000.000000,2000.000000,2000.000000
mean,60.451000,120.328950,25.188000
std,11.544229,10.180132,2.982409
min,40.000000,76.600000,15.500000
25%,51.000000,113.675000,23.200000
50%,60.000000,120.600000,25.100000
75%,70.000000,127.200000,27.200000
max,80.000000,152.200000,35.600000


About 25% of the 2,000 individuals die – this is a high-risk population. There is roughly 50% males and
50% females, aged 40-80.

### Split data into training and test sets

In [35]:
# factorize binary variables to 1 and 0
data_predict['sex'].replace({'Male': 1, 'Female': 0}, inplace=True)
data_predict['dead'].replace({'Dead': 1, 'Alive': 0}, inplace=True)

data_train = data_predict.sample(frac=0.5, random_state=777)
data_test = data_predict.drop(data_train.index)

In [36]:
data_train

,id,age,sbp,bmi,sex,dead
563,564,78.0,134.6,23.4,1,1
892,893,46.0,101.5,18.3,0,0
827,828,49.0,131.4,23.7,1,0
316,317,69.0,112.5,26.2,0,0
1968,1969,52.0,124.5,24.8,1,0
...,...,...,...,...,...,...
213,214,79.0,111.4,22.1,0,1
667,668,79.0,111.0,22.7,0,0
1621,1622,70.0,128.0,26.6,0,1
1455,1456,63.0,120.9,22.5,1,0


In [37]:
data_test

,id,age,sbp,bmi,sex,dead
0,1,47.0,116.8,25.6,0,0
2,3,41.0,130.7,25.6,1,0
4,5,54.0,120.3,20.7,1,0
6,7,71.0,129.3,32.9,0,1
7,8,73.0,117.4,27.5,1,0
...,...,...,...,...,...,...
1992,1993,64.0,126.2,20.0,0,0
1993,1994,68.0,113.5,22.9,1,1
1994,1995,59.0,135.5,34.3,1,0
1996,1997,79.0,131.2,33.0,1,0


### Fit logistic regression model using the training data

In [38]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state = 777, penalty = 'none')

In [39]:
train_vars = ['age', 'sbp', 'bmi', 'sex']

model = logreg.fit(data_train[train_vars], data_train['dead'])

In [40]:
# Get coefficients of training variables
pd.DataFrame.from_records(model.coef_, columns = train_vars, index = ['coef'])

,age,sbp,bmi,sex
coef,0.086079,0.047468,0.031617,0.16342


,age,sbp,bmi,sex
coef,0.086078,0.047489,0.031541,-0.163535
